# Analyse de l'accessibilité piétonnière avec Python

Nous allons reproduire les [analyses de l'article de Gispo Ltd](https://gispofinland.medium.com/analysing-urban-walkability-using-openstreetmap-and-python-33815d045204). Cet article traite de l'évaluation de l'accessibilité piétionnière dans la ville. Il met à disposition son code dans [ce repo Github](https://github.com/eemilhaa/walkability-analysis). Il prend l'exemple de la Finlande, mais nous allons prendre un exemple en France, la ville de Lyon. L'idée est d'utiliser des outils et données libres, afin de réaliser une analyse en deux partie :

- la mesure de l'accessibilité globale via la densité d'intersection,
- la mesure de l'accessibilité des équipements au sein d'une ville.

Le sujet de l'accessibilité piétonne est associé à d'autres thématiques :
- présence d'espace vert,
- qualité de l'air,
- pollution par voiture etc.

## 1/ Cadre théorique

### 1.1/ La densité des intersections

La densité d'intersection est un indicateur sur l'interconnexion du réseau. Une [corrélation positive entre la densité d'intersection et l'usage de la marche comme principal mode de transport a été démontrée dans cet article](https://www.tandfonline.com/doi/abs/10.1080/01944361003766766). Ce qui renforce l'intérêt de cet indicateur dans la compréhension de l'accessibilité de l'espace urbain.

**Remarque**: au cours de son explication, l'analyste indique avoir réalisé des traitements "grossièrement", sans tenir compte de chaque spécificité. Par exemple, quand eux rues fusionnent en une à des points différents, cela peut représenter deux intersections. Ce qui revient à sur-évaluer le nombre d'intersection sur cette rue fusionnée. On peut donc dire qu'il s'agit surtout de donner une idée globale, d'une situation, sans aller trop dans le détail, afin de trouver le juste milieu entre qualité et rapidité.



In [ ]:
## import des modules